# 💾 ARMAZENAMENTO QDRANT

Insere embeddings no banco vetorial Qdrant.

In [1]:
import os
import json
import hashlib
from pathlib import Path
from datetime import datetime
from collections import defaultdict
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance, VectorParams, PointStruct,
    Filter, FieldCondition, MatchValue, FilterSelector
)

# Ler commit do contexto salvo pelo notebook 2
try:
    with open("pipeline_data/context.json", "r") as f:
        context = json.load(f)
    GITLAB_COMMIT = context["gitlab_commit"]
    print(f"📍 Commit lido do contexto: {GITLAB_COMMIT[:8]}")
except:
    GITLAB_COMMIT = "unknown"
    print("⚠️ Contexto não encontrado, usando commit 'unknown'")

# Configuração
QDRANT_URL = os.getenv("QDRANT_URL", "http://qdrant.codrstudio.dev:6333")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
COLLECTION_NAME = os.getenv("QDRANT_COLLECTION", "nic")

# Configuração do GitLab (para metadados)
GITLAB_REPO = "nic/documentacao/base-de-conhecimento"
GITLAB_BRANCH = os.getenv("GITLAB_BRANCH", "main")

# Versionamento
EMBED_MODEL_MAJOR = "v1"  # Versão major do modelo
EMBED_MODEL_FULL = "BAAI/bge-m3"
TOKENIZER_MAJOR = "v1"
TOKENIZER_FULL = "RecursiveCharacterTextSplitter-500-100"

if not QDRANT_API_KEY:
    raise ValueError("QDRANT_API_KEY é obrigatório")

# Diretórios
embeddings_dir = Path("pipeline_data/embeddings")

print(f"Qdrant URL: {QDRANT_URL}")
print(f"Collection: {COLLECTION_NAME}")
print(f"Repository: {GITLAB_REPO}")
print(f"Branch: {GITLAB_BRANCH}")
print(f"API Key: ***{QDRANT_API_KEY[-4:] if len(QDRANT_API_KEY) > 4 else '***'}")

def calculate_content_hash(text: str) -> str:
    """Calcula hash SHA256 do conteúdo"""
    return hashlib.sha256(text.encode('utf-8')).hexdigest()

def generate_deterministic_id(repo: str, relpath: str, chunk_index: int, 
                            tokenizer_major: str, embed_model_major: str) -> str:
    """Gera ID determinístico para pontos do Qdrant"""
    # Combinar todos os elementos que tornam o chunk único
    id_string = f"{repo}:{relpath}:{chunk_index}:{tokenizer_major}:{embed_model_major}"
    
    # Usar hash para gerar ID numérico determinístico
    hash_object = hashlib.sha256(id_string.encode('utf-8'))
    # Converter para inteiro usando os primeiros 8 bytes do hash
    return int.from_bytes(hash_object.digest()[:8], byteorder='big')

📍 Commit lido do contexto: e9c8a430
Qdrant URL: http://qdrant.codrstudio.dev:6333
Collection: nic
Repository: nic/documentacao/base-de-conhecimento
Branch: main
API Key: ***d857


In [2]:
# Conectar ao Qdrant
client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY
)

# Verificar conexão
collections = client.get_collections()
print(f"✅ Conectado ao Qdrant")
print(f"Collections existentes: {len(collections.collections)}")

for col in collections.collections:
    print(f"  - {col.name}")

✅ Conectado ao Qdrant
Collections existentes: 3
  - documents
  - nic_storage
  - nic


/tmp/ipykernel_942480/3416817368.py:2: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(


In [3]:
# Carregar embeddings
embeddings_file = embeddings_dir / "embeddings.jsonl"

if not embeddings_file.exists():
    raise FileNotFoundError(f"Arquivo de embeddings não encontrado: {embeddings_file}")

embeddings_data = []
with open(embeddings_file, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        embeddings_data.append(data)

print(f"Embeddings carregados: {len(embeddings_data)}")

# Verificar dimensões
if embeddings_data:
    embedding_dim = len(embeddings_data[0]["embedding"])
    print(f"Dimensões do embedding: {embedding_dim}")
else:
    raise ValueError("Nenhum embedding encontrado")

# Enriquecer com metadados completos
timestamp_now = datetime.now().isoformat()

for embedding in embeddings_data:
    # Adicionar metadados expandidos
    embedding["metadata"] = {
        # Identificação do repositório
        "repo": GITLAB_REPO,
        "branch": GITLAB_BRANCH,
        "relpath": embedding["source_document"],  # Caminho relativo
        "commit": GITLAB_COMMIT,
        "last_updated": timestamp_now,
        
        # Versionamento de embeddings
        "embed_model_major": EMBED_MODEL_MAJOR,
        "embed_model_full": EMBED_MODEL_FULL,
        "tokenizer_major": TOKENIZER_MAJOR,
        "tokenizer_full": TOKENIZER_FULL,
        
        # Hashes para validação
        "content_sha256": calculate_content_hash(embedding["text"]),
        # doc_sha256 seria o hash do documento completo (implementar se necessário)
        
        # Outros metadados
        "lang": "pt-BR",  # Português do Brasil
        "processing_date": timestamp_now,
        "pipeline_version": "1.0.0"
    }

print(f"✅ Metadados adicionados a todos os embeddings")

# Exemplo de metadados
if embeddings_data:
    print(f"\nExemplo de metadados:")
    sample = embeddings_data[0]["metadata"]
    for key, value in sample.items():
        print(f"  {key}: {value}")
        if len(str(value)) > 50:  # Truncar valores longos
            break

Embeddings carregados: 322
Dimensões do embedding: 1024
✅ Metadados adicionados a todos os embeddings

Exemplo de metadados:
  repo: nic/documentacao/base-de-conhecimento
  branch: main
  relpath: 30-Aprovados/Mapas/Visão Geral do Self Checkout
  commit: e9c8a430b8bc05c306cc8fb342f42e7b45a18744
  last_updated: 2025-08-18T02:32:41.008968
  embed_model_major: v1
  embed_model_full: BAAI/bge-m3
  tokenizer_major: v1
  tokenizer_full: RecursiveCharacterTextSplitter-500-100
  content_sha256: 369066d281dea4e27277770c635a21cc61edf93e0a73a18d5ef5b26f59ce3edf


In [4]:
# Verificar/criar collection
collection_exists = False
try:
    collection_info = client.get_collection(COLLECTION_NAME)
    collection_exists = True
    print(f"\n📊 Collection '{COLLECTION_NAME}' já existe:")
    print(f"  Pontos: {collection_info.points_count}")
    print(f"  Status: {collection_info.status}")
    
    # Em modo FULL, vamos reindexar tudo
    print(f"\n🔄 Modo FULL: Reindexação completa será realizada")
    print(f"  Arquivos existentes serão substituídos atomicamente")
    
except Exception:
    print(f"Collection '{COLLECTION_NAME}' não existe, criando...")

if not collection_exists:
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(
            size=embedding_dim,
            distance=Distance.COSINE
        )
    )
    print(f"✅ Collection '{COLLECTION_NAME}' criada")
    print(f"  Dimensões: {embedding_dim}")
    print(f"  Distância: COSINE")
else:
    print(f"✅ Collection '{COLLECTION_NAME}' verificada")


📊 Collection 'nic' já existe:
  Pontos: 644
  Status: green

🔄 Modo FULL: Reindexação completa será realizada
  Arquivos existentes serão substituídos atomicamente
✅ Collection 'nic' verificada


In [5]:
# Agrupar embeddings por arquivo para reindexação atômica
chunks_by_file = defaultdict(list)

for embedding_data in embeddings_data:
    source_doc = embedding_data["source_document"]
    chunks_by_file[source_doc].append(embedding_data)

print(f"📁 Arquivos únicos: {len(chunks_by_file)}")
print(f"📊 Total de chunks: {sum(len(chunks) for chunks in chunks_by_file.values())}")
print(f"\n📈 Distribuição de chunks por arquivo:")

# Mostrar estatísticas
for doc, chunks in sorted(chunks_by_file.items(), 
                          key=lambda x: len(x[1]), reverse=True)[:5]:
    print(f"  {doc}: {len(chunks)} chunks")
    
if len(chunks_by_file) > 5:
    print(f"  ... e mais {len(chunks_by_file) - 5} arquivos")

# Função de reindexação por arquivo
def reindex_file(client, collection_name, source_document, chunks_data):
    """
    Reindexação atômica por arquivo com metadados completos.
    Implementa a diretriz: DELETE por filtro + UPSERT com IDs determinísticos
    """
    # PASSO 1: Deletar chunks existentes do arquivo
    try:
        # Filtro para deletar apenas chunks deste arquivo específico
        delete_filter = Filter(
            must=[
                FieldCondition(key="repo", match=MatchValue(value=GITLAB_REPO)),
                FieldCondition(key="branch", match=MatchValue(value=GITLAB_BRANCH)),
                FieldCondition(key="relpath", match=MatchValue(value=source_document))
            ]
        )
        
        delete_result = client.delete(
            collection_name=collection_name,
            points_selector=FilterSelector(filter=delete_filter)
        )
        # Não mostrar mensagem se não deletou nada (primeira execução)
        
    except Exception as e:
        # Ignorar erro se não há dados para deletar
        pass
    
    # PASSO 2: Preparar novos pontos com IDs determinísticos e metadados completos
    points = []
    for chunk_data in chunks_data:
        # Gerar ID determinístico
        point_id = generate_deterministic_id(
            repo=GITLAB_REPO,
            relpath=chunk_data["source_document"],
            chunk_index=chunk_data["chunk_index"],
            tokenizer_major=TOKENIZER_MAJOR,
            embed_model_major=EMBED_MODEL_MAJOR
        )
        
        # Payload completo com todos os metadados
        payload = {
            # Dados originais do chunk
            "chunk_id": chunk_data["chunk_id"],
            "chunk_index": chunk_data["chunk_index"],
            "text": chunk_data["text"],
            "char_count": chunk_data["char_count"],
            
            # Metadados expandidos (todos os campos da diretriz)
            "repo": GITLAB_REPO,
            "branch": GITLAB_BRANCH,
            "relpath": chunk_data["source_document"],
            "source_document": chunk_data["source_document"],  # Manter compatibilidade
            "commit": GITLAB_COMMIT,
            "last_updated": chunk_data["metadata"]["last_updated"],
            
            # Versionamento
            "embed_model_major": EMBED_MODEL_MAJOR,
            "embed_model_full": EMBED_MODEL_FULL,
            "tokenizer_major": TOKENIZER_MAJOR,
            "tokenizer_full": TOKENIZER_FULL,
            "embedding_model": chunk_data["embedding_model"],  # Manter compatibilidade
            
            # Hashes
            "content_sha256": chunk_data["metadata"]["content_sha256"],
            
            # Outros
            "lang": "pt-BR",
            "processing_date": chunk_data["metadata"]["processing_date"],
            "pipeline_version": chunk_data["metadata"]["pipeline_version"]
        }
        
        point = PointStruct(
            id=point_id,
            vector=chunk_data["embedding"],
            payload=payload
        )
        points.append(point)
    
    # PASSO 3: Upsert (inserir ou atualizar) novos pontos
    if points:
        client.upsert(
            collection_name=collection_name,
            points=points,
            wait=True  # Esperar confirmação
        )
    
    return len(points)

# Teste da função com um arquivo
test_file = list(chunks_by_file.keys())[0]
test_chunks = chunks_by_file[test_file][:1]  # Apenas 1 chunk para teste

print(f"\n🧪 Preparando reindexação:")
print(f"  Exemplo ID determinístico: {generate_deterministic_id(GITLAB_REPO, test_file, 0, TOKENIZER_MAJOR, EMBED_MODEL_MAJOR)}")
print(f"  Arquivo de teste: {test_file}")

📁 Arquivos únicos: 31
📊 Total de chunks: 322

📈 Distribuição de chunks por arquivo:
  30-Aprovados/Mapas/Visão Geral do Self Checkout: 51 chunks
  30-Aprovados/Tópicos/Componentes principais do sistema: 19 chunks
  30-Aprovados/Tópicos/Pré-requisitos técnicos: 17 chunks
  30-Aprovados/Mapas/Visão Geral do NIC: 15 chunks
  30-Aprovados/Tópicos/Padrões de Documentação do NIC: 15 chunks
  ... e mais 26 arquivos

🧪 Preparando reindexação:
  Exemplo ID determinístico: 4458554930477711609
  Arquivo de teste: 30-Aprovados/Mapas/Visão Geral do Self Checkout


In [6]:
# Modo FULL: Reindexar todos os arquivos
print(f"🚀 INICIANDO REINDEXAÇÃO FULL")
print(f"  Modo: DELETE + UPSERT por arquivo")
print(f"  Repo: {GITLAB_REPO}")
print(f"  Branch: {GITLAB_BRANCH}")
print(f"  Commit: {GITLAB_COMMIT}")
print("=" * 60)

total_inserted = 0
total_files = len(chunks_by_file)
errors = []

for idx, (source_document, file_chunks) in enumerate(chunks_by_file.items(), 1):
    try:
        # Reindexar arquivo
        inserted = reindex_file(
            client=client,
            collection_name=COLLECTION_NAME,
            source_document=source_document,
            chunks_data=file_chunks
        )
        
        total_inserted += inserted
        
        # Progress bar simples
        progress = idx / total_files * 100
        bar = "█" * int(progress / 5) + "░" * (20 - int(progress / 5))
        print(f"\r[{bar}] {progress:.1f}% - {idx}/{total_files} arquivos", end="")
        
    except Exception as e:
        errors.append((source_document, str(e)))

print(f"\n" + "=" * 60)
print(f"✅ REINDEXAÇÃO COMPLETA!")
print(f"  📁 Arquivos processados: {total_files}")
print(f"  📊 Chunks inseridos: {total_inserted}")
print(f"  ❌ Erros: {len(errors)}")

if errors:
    print(f"\n⚠️ Arquivos com erro:")
    for doc, error in errors[:5]:
        print(f"  {doc}: {error}")

# Salvar metadados da inserção no contexto  
stage_metadata = {
    "files_processed": total_files,
    "chunks_inserted": total_inserted,
    "errors_count": len(errors),
    "qdrant_collection": COLLECTION_NAME,
    "reindex_mode": "FULL",
    "gitlab_commit": GITLAB_COMMIT,
    "pipeline_versions": {
        "embed_model_major": EMBED_MODEL_MAJOR,
        "embed_model_full": EMBED_MODEL_FULL,
        "tokenizer_major": TOKENIZER_MAJOR,
        "tokenizer_full": TOKENIZER_FULL
    },
    "errors": errors[:10] if errors else []  # Limitar erros salvos
}

# Salvar no arquivo de contexto
with open("pipeline_data/context.json", "r") as f:
    context_data = json.load(f)

context_data["stage_06_armazenamento_qdrant"] = stage_metadata

with open("pipeline_data/context.json", "w") as f:
    json.dump(context_data, f, indent=2, ensure_ascii=False)

print(f"\n✅ Metadados da inserção salvos no contexto do stage")

🚀 INICIANDO REINDEXAÇÃO FULL
  Modo: DELETE + UPSERT por arquivo
  Repo: nic/documentacao/base-de-conhecimento
  Branch: main
  Commit: e9c8a430b8bc05c306cc8fb342f42e7b45a18744


[███░░░░░░░░░░░░░░░░░] 16.1% - 5/31 arquivos

[███░░░░░░░░░░░░░░░░░] 19.4% - 6/31 arquivos

[███████░░░░░░░░░░░░░] 35.5% - 11/31 arquivos

[██████████░░░░░░░░░░] 54.8% - 17/31 arquivos

[███████████░░░░░░░░░] 58.1% - 18/31 arquivos

[██████████████░░░░░░] 71.0% - 22/31 arquivos

[██████████████████░░] 90.3% - 28/31 arquivos

[██████████████████░░] 93.5% - 29/31 arquivos

[████████████████████] 100.0% - 31/31 arquivos
✅ REINDEXAÇÃO COMPLETA!
  📁 Arquivos processados: 31
  📊 Chunks inseridos: 322
  ❌ Erros: 0

✅ Metadados da inserção salvos no contexto do stage
